In [51]:
import pymunk
import pygame
import pymunk.pygame_util

In [52]:
# pygame.init()
# screen = pygame.display.set_mode((600, 600))
# draw_options = pymunk.pygame_util.DrawOptions(screen)
# clock = pygame.time.Clock()
# running = True

# space = pymunk.Space()
# space.gravity = 0, 100
# space.iterations = 300
# space.damping = 0.3 

In [53]:
CATEGORY_1 = 0b001 
CATEGORY_2 = 0b010 
CATEGORY_3 = 0b100 

# Create shapes with appropriate filters
# Object 1: Can collide with categories 2 and 3
road = pymunk.ShapeFilter(
    categories=CATEGORY_1,           
    mask=CATEGORY_2 | CATEGORY_3   
)

# Object 2: Can collide with category 1 only
ball = pymunk.ShapeFilter(
    categories=CATEGORY_2,           # This object is type 2
    mask=CATEGORY_1                  # Can collide with type 1 only
)

# Object 3: Can collide with category 1 only  
support = pymunk.ShapeFilter(
    categories=CATEGORY_3,           # This object is type 3
    mask=CATEGORY_1                  # Can collide with type 1 only
)

In [54]:
class Bridge:
    def __init__(self, space, nodes, edges, static_nodes):
        """
        space: pymunk.Space - the physics space
        nodes: dict - {node_id: (x, y)} coordinates of nodes
        edges: list - {(node_a, node_b): (mass,isRoad)} edges by node ids with mass
        static_nodes: list - node ids that are static with respect to world
        """
        self.space = space
        space.damping = 0.3
        space.iterations = 300

        self.nodes = nodes
        self.edges = edges
        self.static_nodes = static_nodes

        self.bodies = {}
        self.shapes = []
        self.joints = {}

        adj = {} # node_id: [node_id1, ...]

        for (a, b), (mass,isRoad) in self.edges.items():
            if a > b:
                a, b = b, a

            if a not in adj.keys():
                adj[a] = []
            if b not in adj.keys():
                adj[b] = []
            adj[a].append(b)
            adj[b].append(a)

            a_pos = self.nodes[a]
            b_pos = self.nodes[b]

            body = pymunk.Body()
            body.position = (a_pos + b_pos) / 2
            self.space.add(body)
            self.bodies[a, b] = body

            segment = pymunk.Segment(body, a_pos - body.position, b_pos - body.position, radius=5)
            segment.filter = road if isRoad else support
            segment.mass = mass
            self.space.add(segment)
            self.shapes.append(segment)
        
        for a, nbhs in adj.items():
            a_pos = self.nodes[a]

            for i in range(len(nbhs)):
                for j in range(i + 1, len(nbhs)):
                    a1 = a
                    a2 = a
                    b = nbhs[i]
                    c = nbhs[j]

                    if a1 > b:
                        a1, b = b, a1
                    if a2 > c:
                        a2, c = c, a2

                    b_body = self.bodies[a1, b]
                    c_body = self.bodies[a2, c]
                    
                    joint = pymunk.PivotJoint(b_body, c_body, a_pos)
                    joint.collide_bodies = False
                    self.space.add(joint)
                    self.joints[a] = joint

        for a, b in self.edges:
            if a in self.static_nodes:
                stat = a
            elif b in self.static_nodes:
                stat = b
            else:
                continue

            if a > b:
                a, b = b, a

            body = pymunk.Body(body_type=pymunk.Body.STATIC)
            body.position = self.nodes[stat]
            self.space.add(body)

            joint = pymunk.PivotJoint(body, self.bodies[a, b], body.position)
            joint.collide_bodies = False
            self.space.add(joint)
            self.joints[a] = joint


In [55]:
def constant_horizontal_velocity(body, gravity, damping, dt):
    """Custom velocity function to maintain constant horizontal speed"""
    target_horizontal_velocity = 20 
    pymunk.Body.update_velocity(body, gravity, damping, dt)
    body.velocity = (target_horizontal_velocity, body.velocity.y)

def simulate(space,position):
    body = pymunk.Body(mass=100,moment=10)
    body.position = position
    body.velocity_func = constant_horizontal_velocity
    shape = pymunk.Circle(body,20)
    shape.filter = ball
    space.add(body,shape)

    

In [56]:
# a1  = pymunk.Body(body_type=pymunk.Body.STATIC)
# a1.position = 100,500
# c1 = pymunk.Circle(a1, radius=5)
# c1.friction = 0.7 
# c1.elasticity = 0.1
# a2  = pymunk.Body(body_type=pymunk.Body.STATIC)
# a2.position = 500,500
# c2 = pymunk.Circle(a2, radius=5)
# c2.friction = 0.7
# c2.elasticity = 0.1
# space.add(a1,c1,a2,c2)

# roads = []
# joints = []  # Store joints to access their forces later

# for x in range(0, 4):
#     r = pymunk.Body(mass=5, moment=5)
#     r.position = 150 + x * 100, 500
#     print(r.position)
#     rod = pymunk.Segment(r, (-51, 0), (51, 0), 10)
#     rod.friction = 0.8
#     rod.elasticity = 0.1  
    
#     if(x == 0):
#         joint = pymunk.constraints.PivotJoint(a1, r, (0, 0), (-51, 0))
#     else:
#         joint = pymunk.constraints.PivotJoint(roads[-1], r, (51, 0), (-51, 0))

#     joint.collide_bodies = False
#     roads.append(r)
#     joints.append(joint)  # Store joint reference
#     space.add(r, rod, joint)


# final_joint = pymunk.constraints.PivotJoint(roads[-1], a2, (51, 0), (0, 0))
# final_joint.collide_bodies = False
# joints.append(final_joint)  
# space.add(final_joint)

In [57]:
# Define nodes and edges
nodes = {
    1: pymunk.Vec2d(100, 200),
    2: pymunk.Vec2d(200, 200),
    3: pymunk.Vec2d(300, 200),
    4: pymunk.Vec2d(400, 200),
    5: pymunk.Vec2d(150, 100),
    6: pymunk.Vec2d(250, 100),
    7: pymunk.Vec2d(350, 100),   
}

edges = {
    (1, 2): (10,1),
    (2, 3): (10,1),
    (3, 4): (10,1),

    (1, 5): (10,0),
    (2, 5): (10,0),
    (2, 6): (10,0),
    (3, 6): (10,0),
    (3, 7): (10,0),
    (4, 7): (10,0),

    (5, 6): (10,0),
    (6, 7): (10,0),
    
}

static_nodes = [1, 7]

In [58]:
# bridge = Bridge(space, nodes, edges, static_nodes)

# dt = 0.01

# isCdown = 0

# while running:
#     for event in pygame.event.get():
#         if event.type == pygame.QUIT:
#             running = False

#     screen.fill("white")
#     space.debug_draw(draw_options)
    
#     keys = pygame.key.get_pressed()
#     if keys[pygame.K_c] and not isCdown:
#         isCdown = 1
#         simulate(space,(111,50))
#     if isCdown and not keys[pygame.K_c]:
#         isCdown = 0



#     space.step(dt)
    
#     # Get tension for each joint
#     for i, joint in bridge.joints.items():
#         tension = joint.impulse / dt
#         print(f"Segment {i} tension: {tension:.2f}N")
    
#     pygame.display.update()

# pygame.quit()

In [59]:
def getfitness(nodes, edges, static_nodes , start_pos , end_x):
    space = pymunk.Space()
    space.gravity = 0, 100
    space.iterations = 300
    space.damping = 0.3 

    bridge = Bridge(space, nodes, edges, static_nodes)

    dt = 0.01


    body = pymunk.Body(mass=10,moment=10)
    body.position = start_pos
    body.velocity_func = constant_horizontal_velocity
    shape = pymunk.Circle(body,20)
    shape.filter = ball
    space.add(body,shape)
    
    max_tension = 0

    while body.position.x <= end_x:       
        space.step(dt)

        for i, joint in bridge.joints.items():
            tension = joint.impulse / dt
            max_tension = max(max_tension,tension)
    
    return max_tension
   


In [60]:
getfitness(nodes,edges,static_nodes,(100,180),500)

198331.75220381067